In [1]:
import uproot
import awkward as ak
from pathlib import Path
import time
import matplotlib.pyplot as plt
!pip3 install requests aiohttp

Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [9]:
data_dir = Path("/project/etp1/dkoch/ATLASOpenData-test")
samples_new = {
    "ttbar": {
        "list": [
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000001.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000002.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000003.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000004.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000005.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000008.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000009.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000010.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000011.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000012.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000013.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000014.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000016.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000017.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000018.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000019.output_ntup.root",
            data_dir / "ttbar-samples-with-weights/user.garciarm.39227472._000020.output_ntup.root",
        ]
    }
}
treename_new = "analysis"
branches_new = [
    "lep_pt","lep_eta","lep_phi",
    "lep_e","lep_charge","lep_type", "lep_n",
    "mcWeight",
    # "scaleFactor_PILEUP",
    # "scaleFactor_ELE",
    # "scaleFactor_MUON",
]

samples_old = {
    "ttbar": {
        "list": [
            "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_410011.single_top_tchan.2lep.root",
        ]
    }
}
treename_old = "mini"
branches_old = [
    "lep_pt","lep_eta","lep_phi",
    "lep_E","lep_charge","lep_type", "lep_n",
    "mcWeight", "scaleFactor_PILEUP",
    "scaleFactor_ELE","scaleFactor_MUON",
    "scaleFactor_LepTRIGGER",
]

In [10]:
import uproot
import numpy as np
import time

def get_new_data(samples, treename, branches):
    data = {}
    for s in samples:
        print(f"Processing {s} samples")
        chunks = []
        try:
            for chunk in uproot.iterate({file: treename for file in samples[s]["list"]}, expressions=branches):
                n_in = len(chunk)
                chunk = chunk[chunk["lep_n"] >= 2]
                n_out = len(chunk)
                print(f"Processed chunk: {n_in} -> {n_out}")
                chunks.append(chunk)
        except TimeoutError as e:
            print(f"TimeoutError while processing {s} samples: {e}")
        except Exception as e:
            print(f"An error occurred while processing {s} samples: {e}")
        if chunks:
            data[s] = np.concatenate(chunks)
    return data

# Example usage:
# start = time.time()
# data_old = get_new_data(samples_old, treename_old, branches_old)
# end = time.time()
# print(f"took {end - start}s")


In [4]:
start = time.time()
data_new = get_new_data(samples_new, treename_new, branches_new)
end = time.time()
print(f"took {end - start}s")

Processing ttbar samples


KeyboardInterrupt: 

In [11]:
start = time.time()
data_old = get_new_data(samples_old, treename_old, branches_old)
end = time.time()
print(f"took {end - start}s")

Processing ttbar samples
An error occurred while processing ttbar samples: a bytes-like object is required, not 'TimeoutError'
took 301.52126026153564s


In [12]:
plt.figure(figsize=(12, 9))
for sample in samples_new:
    plt.hist(ak.flatten(data_new[sample].lep_e), range=(0, 800), bins=50, label=f"{sample} new", density=True, histtype="step")
    plt.hist(ak.flatten(data_old[sample].lep_E)/1000, range=(0, 800), bins=50, label=f"{sample} old", density=True, histtype="step")
plt.legend()
plt.title("lep_e, no weights")
plt.xlabel("E [GeV]")
plt.show()

NameError: name 'data_new' is not defined

<Figure size 864x648 with 0 Axes>